factory_boy
==

Il s'agit d'un module permettant de créer des objets à la volée en les remplissant de manière aléatoire et réaliste.

Cet outil permet de faciliter grandement les tests

In [1]:
# You should execute this line to install pydantic and factory_boy
import subprocess
print(subprocess.getstatusoutput("pip install pydantic"))
print(subprocess.getstatusoutput("pip install factory_boy"))

(0, 'Requirement already satisfied: pydantic in /home/sch-bm/Documents/repositories/python_support/.venv/lib/python3.11/site-packages (2.11.5)\nRequirement already satisfied: annotated-types>=0.6.0 in /home/sch-bm/Documents/repositories/python_support/.venv/lib/python3.11/site-packages (from pydantic) (0.7.0)\nRequirement already satisfied: pydantic-core==2.33.2 in /home/sch-bm/Documents/repositories/python_support/.venv/lib/python3.11/site-packages (from pydantic) (2.33.2)\nRequirement already satisfied: typing-extensions>=4.12.2 in /home/sch-bm/Documents/repositories/python_support/.venv/lib/python3.11/site-packages (from pydantic) (4.13.2)\nRequirement already satisfied: typing-inspection>=0.4.0 in /home/sch-bm/Documents/repositories/python_support/.venv/lib/python3.11/site-packages (from pydantic) (0.4.1)')
(0, 'Collecting factory_boy\n  Downloading factory_boy-3.3.3-py2.py3-none-any.whl (37 kB)\nCollecting Faker>=0.7.0\n  Downloading faker-37.3.0-py3-none-any.whl (1.9 MB)\n\x1b[?2

In [2]:
from pydantic import BaseModel
from enum import Enum, auto
from datetime import date
from uuid import UUID

class TypeVoie(Enum):
    _ = auto()
    bis = auto()
    ter = auto()

class Adresse(BaseModel):
    pk: int
    numero: int
    type_voie: TypeVoie
    rue: str
    code_postal: str
    ville: str

class Personne(BaseModel):
    pk: UUID
    prenom: str
    nom: str
    date_naissance: date
    adresse: Adresse

In [3]:
import factory
from uuid import uuid4

class AdresseFactory(factory.Factory):
    class Meta:
        model = Adresse

    pk = factory.Sequence(lambda x: 10000 + x)
    numero = factory.Faker("pyint", min_value=1, max_value=250)
    type_voie = factory.Iterator(TypeVoie)
    rue = factory.Faker("street_name", locale="fr_FR")
    code_postal = factory.Faker("postcode", locale="fr_FR")
    ville = factory.Faker("city", locale="fr_FR")

class PersonneFactory(factory.Factory):
    class Meta:
        model = Personne

    pk = factory.LazyFunction(uuid4)
    prenom = factory.Faker("first_name", locale="fr_FR")
    nom = factory.Faker("last_name", locale="fr_FR")
    date_naissance = factory.Faker(
        'date_between_dates',
        date_start=date(1980, 1, 1),
        date_end=date(2008, 12, 31),
    )
    adresse = factory.SubFactory(AdresseFactory)

In [4]:
adresse = AdresseFactory()

In [5]:
adresse

Adresse(pk=10000, numero=207, type_voie=<TypeVoie._: 1>, rue='chemin de Pruvost', code_postal='08508', ville='Sainte Hortense-les-Bains')

In [6]:
AdresseFactory.create()

Adresse(pk=10001, numero=195, type_voie=<TypeVoie.bis: 2>, rue='chemin de Lucas', code_postal='65867', ville='Sainte Joséphine')

In [7]:
AdresseFactory.create_batch(5)

[Adresse(pk=10002, numero=181, type_voie=<TypeVoie.ter: 3>, rue='rue de Colas', code_postal='91655', ville='David'),
 Adresse(pk=10003, numero=24, type_voie=<TypeVoie._: 1>, rue='rue Maréchal', code_postal='66472', ville='Sainte Anaïs-sur-Mer'),
 Adresse(pk=10004, numero=153, type_voie=<TypeVoie.bis: 2>, rue='chemin Poulain', code_postal='48465', ville='Andre'),
 Adresse(pk=10005, numero=219, type_voie=<TypeVoie.ter: 3>, rue='chemin de Verdier', code_postal='54520', ville='Lévêque'),
 Adresse(pk=10006, numero=185, type_voie=<TypeVoie._: 1>, rue='avenue de Thomas', code_postal='17301', ville='Rivièrenec')]

In [8]:
AdresseFactory.create(ville="Montpellier")

Adresse(pk=10007, numero=63, type_voie=<TypeVoie.bis: 2>, rue='rue Céline Marin', code_postal='69509', ville='Montpellier')

In [9]:
PersonneFactory.create()

Personne(pk=UUID('4cd7f946-2dc5-4882-b978-8985c4c09d9c'), prenom='Agnès', nom='Colin', date_naissance=datetime.date(2006, 9, 28), adresse=Adresse(pk=10008, numero=59, type_voie=<TypeVoie.ter: 3>, rue='avenue de Daniel', code_postal='89797', ville='Duboisdan'))

In [10]:
PersonneFactory.create(adresse=adresse, prenom="John", nom="Doe")

Personne(pk=UUID('b54465c3-26db-4bb6-9a27-9f7f3c0d8326'), prenom='John', nom='Doe', date_naissance=datetime.date(1984, 5, 14), adresse=Adresse(pk=10000, numero=207, type_voie=<TypeVoie._: 1>, rue='chemin de Pruvost', code_postal='08508', ville='Sainte Hortense-les-Bains'))

In [12]:
import unittest

class TestOrder(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        cls.adresses = AdressesFactory.create_batch(10)

    def setUp(self):
        super().setUp()
        self.personne = PersonneFactory()

    def tearDown(self):
        super().tearDown()

    @classmethod
    def tearDownClass(self):
        super().tearDownClass()

    def test_create_order_with_this(self): ...

    def test_create_order_with_that(self): ...


Documentation :

* https://factoryboy.readthedocs.io/en/stable/examples.html
* https://faker.readthedocs.io/en/stable/providers.html

---